# PostgreSQL Data Interaction with psycopg2 and pandas

In [2]:
import psycopg2
import pandas as pd

PostgreSQL Database Creation and Connection:

In [3]:
def create_database():
    # Connect to the PostgreSQL server
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    cur = conn.cursor()
    conn.set_session(autocommit=True)
    
    # Drop the existing database named "db" if it exists
    cur.execute("DROP DATABASE db")
    # Create a new database named "db"
    cur.execute("CREATE DATABASE db")
    
    # Close the connection to the default database
    conn.close()
    
    # Connect to the new database "db"
    conn = psycopg2.connect("host=127.0.0.1 dbname=db user=postgres password=root")
    cur = conn.cursor()
    
    return cur, conn

# Loading title.csv Data:

In [4]:
Moviedf = pd.read_csv("titles.csv")

In [5]:
Moviedf.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


# Simplifying Data: Selecting only key Columns 

In [6]:
titledf = Moviedf[['id', 'title', 'type', 'description', 'release_year']]

In [7]:
titledf.head()

,id,title,type,description,release_year
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967


# Loading credit.csv Data:

In [8]:
creditdf = pd.read_csv("credits.csv")

In [9]:
creditdf.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [10]:
cur, conn = create_database()

# Creating and Structuring the 'titledf' & 'creditdf' Table in PostgreSQL

In [11]:
movies = ("CREATE TABLE IF NOT EXISTS titledf \
(id VARCHAR(10) PRIMARY KEY, title VARCHAR(255), type VARCHAR(50), description TEXT, release_year INTEGER)")
cur.execute(movies)
conn.commit()

In [12]:
person = ("CREATE TABLE IF NOT EXISTS creditdf \
(person_id INTEGER PRIMARY KEY, id VARCHAR(10), name VARCHAR(30), character VARCHAR(250), role VARCHAR(10))")
cur.execute(person)
conn.commit()

In [13]:
conn.commit()

# Inserting Data into 'titledf' & 'creditdf' Table 

In [14]:
movies_insert = ("""INSERT INTO titledf(
id, title, type, description, release_year)
VALUES (%s, %s, %s, %s, %s)""")

In [15]:
for i, row in titledf.iterrows():
    cur.execute(movies_insert, list(row))

In [16]:
person_insert = ("""INSERT INTO creditdf(
person_id, id, name, character, role)
VALUES (%s, %s, %s, %s, %s)""")

In [17]:
for i, row in creditdf.iterrows():
    cur.execute(person_insert, list(row))

In [18]:
conn.commit()